In [1]:
import sys
import time
project_path = r'D:/dev/Python/SVNTrace/SVNTraceAddRVOMS'
sys.path.append(project_path)
sys.path.append(project_path+'/repos/ClangParserProject')
sys.path.append(project_path + r'/repos/PyUtil')



In [12]:
from svn_oms.parser2 import SVNProjectParser

path = r'D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation'
start_rv = '7990'
end_rv = '8062'
#end는 head0

st_t = time.time()
parser = SVNProjectParser(project_path=path, start_rv=start_rv, end_rv=end_rv)
result = parser.parse()
ed_t = time.time()

print('time:', ed_t - st_t)

start update end rv
end update end rv
(9 / 10) 8062	10) done. 23 change file times 	
parse D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h	braryDrawer.h				p				passpasspasspasspasspasspasspasspasspass
End base Set
parse D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h	p	pp	p	asspasspasspasspasspasspasspasspasspasspasspasspasspasspasspasspasspasspasspasspassparse 120.17 sec
time: 120.16584539413452


In [13]:
from clangParser.datas.CUnit import CUnit, Cursor
from oms.dataset.info_base import InfoBase
from oms.dataset.function_info import FunctionInfo
import oms.Mapper as Mapper

from collections import defaultdict

# unit = CUnit.parse(r'D:\dev\AutoPlanning\trunk\Ap-Trunk-auto-task\DoitRenderer\DirectXRenderer.cpp')


#step 1. rv Unit - InfoBase 생성
# InfoBase 관련 상세 설명:
# 1. InfoBase는 각 C/C++ 코드 단위(함수, 변수, 클래스 등)의 기본 정보를 저장하는 객체입니다.
# 2. Mapper.Cursor2InfoBase 함수를 통해, CUnit의 Cursor 정보를 파싱하여 InfoBase 객체로 변환됩니다.
# 3. 생성된 InfoBase 객체는 단위 식별자(src_name)와 함께, 해당 소스 파일의 경로, 수정 이력 등 메타 데이터를 포함합니다.
# 4. 특히, FunctionInfo (InfoBase의 하위 객체)의 경우, 동일 코드 수정 시 파일 확장자(.cpp, .h)에 따라 중복 처리를 방지합니다.

rv_src_info_map :dict[int, dict[str, InfoBase]] = defaultdict(dict)

for trace_data in parser.trace_data_map.rv_path_trace_map.values():
    rv = trace_data.revision
    path = trace_data.file_path
    #1차로 유닛이 있는 케이스만 생성함 (삭제된 파일은 추후 고려)
    if trace_data.unit: 
        unit:CUnit = trace_data.unit

        child_cursors: list[Cursor] = unit.get_this_Cursor()
        for child in child_cursors:
            info:InfoBase = Mapper.Cursor2InfoBase(child)

            #전처리, 힌트 선언 같은 특수 구문은 info가 생성되지 않을 수 있다.
            if info:
                # 메서드(함수)의 경우, cpp와 h 파일이 동시에 수정된 경우 중복 처리를 피하기 위하여
                # cpp 파일에 우선 적용하는 로직을 수행합니다.
                if isinstance(info, FunctionInfo):
                    key = (rv, path.replace('.h', '.cpp'))
                    if path.endswith('.h') and key in parser.trace_data_map.rv_path_trace_map:
                        print('skip:', path, '->', info.src_name)
                        continue

                info.file_path = path #임시로 추가
                if info:
                    rv_src_info_map[rv][info.src_name] = info


skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> ST_SurfaceDataSet.ST_SurfaceDataSet()
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> ST_SurfaceDataSet.IsSameDataSet(CSurfaceModelItem*, CMatrixMxN)
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> CDSILibraryDrawer.CDSILibraryDrawer(LPCWSTR, bool)
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> CDSILibraryDrawer.CDSILibraryDrawer(LPCWSTR, CActuatorPanoFixture*, CLocalDocAP*, bool)
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> CDSILibraryDrawer.~CDSILibraryDrawer()
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> CDSILibraryDrawer.HitTest(const CPoint&)
skip: D:\dev\AutoPlanning\trunk\auto-task\mod_APImplantSimulation\DSILibraryDrawer.h -> CDSILibraryDrawer.OnMouseEvent(UINT, const CPoint&, const int&,

In [16]:
import svn_oms.dataset.rv_info_factory as RVInfoFactory
from svn_oms.dataset.svn_oms import RvInfoBase

#step 2. InfoBase 2 RvInfoBase
rv_src_rvinfo_map :dict[int, dict[str, RvInfoBase]] = defaultdict(dict)

for rv, src_info_map in rv_src_info_map.items():
    for info in src_info_map.values():
        rv_info = RVInfoFactory.info2Rvinfo(rv, info)
        rv_info.file_path = info.file_path #임시로 추가
        rv_src_rvinfo_map[rv][info.src_name] = rv_info

for rv, infos in rv_src_rvinfo_map.items():
    print(f'rv {rv} size : {len(infos)}')



rv 7990 size : 447
rv 7996 size : 51
rv 8010 size : 106
rv 8031 size : 24
rv 8032 size : 191
rv 8034 size : 53
rv 8047 size : 272
rv 8056 size : 30
rv 8058 size : 106
rv 8062 size : 72


In [17]:
#step 3. 수정된 rvinfo 저장
from neo4j_manager.neo4jHandler import Neo4jHandler
db = Neo4jHandler(uri="bolt://localhost:7687", user="neo4j", database='rvinfoip', password="123456789")
db.print_info()
db.delete_all_nodes()

save_datas = []
relation_datas = []

#가장 마지막 path - trace_data
for trace_data in parser.trace_data_map.rv_path_trace_map.values():
    if trace_data.unit:
        edit_src = set(list(trace_data.added_src_node.keys()) + list(trace_data.del_src_node.keys()))
        rv = trace_data.revision
        for src in edit_src:
            if src in rv_src_rvinfo_map[rv]:
                rv_info = rv_src_rvinfo_map[rv][src]
                save_datas.append(rv_info)

            #visit
            before_trace = trace_data
            while src in before_trace.before_src_trace_map:
                before_trace = before_trace.before_src_trace_map[src]
                before_rv = before_trace.revision

                if src in rv_src_rvinfo_map[before_rv]:
                    before_rv_info = rv_src_rvinfo_map[before_rv][src]
                    relation_datas.append((before_rv_info, rv_info, 'update'))
                    save_datas.append(before_rv_info)
                    rv_info = before_rv_info


        
db.save_data(save_datas)
db.print_info()
db.add_relationship(relation_datas)
db.print_info()


Nodes
Relationships
Nodes
	RvFunctionInfo		:	70 size
	RvClassInfo		:	3 size
Relationships
Nodes
	RvFunctionInfo		:	70 size
	RvClassInfo		:	3 size
Relationships
	update		:	40 size


In [ ]:
# #step 3. 수정된 rvinfo 저장
# from neo4j_manager.neo4jHandler import Neo4jHandler
# db = Neo4jHandler(uri="bolt://localhost:7687", user="neo4j", database='rvinfo', password="123456789")
# db.print_info()
# db.delete_all_nodes()

# save_datas = []
# relation_datas = []

# #가장 마지막 path - trace_data
# for trace_data in parser.trace_data_map.rv_path_trace_map.values():
#     if trace_data.unit:
#         edit_src = set(list(trace_data.added_src_node.keys()) + list(trace_data.del_src_node.keys()))
#         rv = trace_data.revision
#         for src in edit_src:
#             if src in rv_src_rvinfo_map[rv]:
#                 rv_info = rv_src_rvinfo_map[rv][src]
#                 save_datas.append(rv_info)

#             #visit
#             before_trace = trace_data
#             while src in before_trace.before_src_trace_map:
#                 before_trace = before_trace.before_src_trace_map[src]
#                 before_rv = before_trace.revision

#                 if src in rv_src_rvinfo_map[before_rv]:
#                     before_rv_info = rv_src_rvinfo_map[before_rv][src]
#                     relation_datas.append((before_rv_info, rv_info, 'update') )
#                     save_datas.append(before_rv_info)
#                     rv_info = before_rv_info


        
# db.save_data(save_datas)
# db.print_info()
# db.add_relationship(relation_datas)
# db.print_info()


In [18]:
from py2neo import Graph
# 리비전별 src_name
query1 = """
MATCH (n)
WITH n.revision AS revision, collect(DISTINCT n.src_name) AS src_names
RETURN revision, src_names
ORDER BY revision
"""

# src_name별 리비전
query2 = """
MATCH (n)
WITH n.src_name AS src_name, collect(DISTINCT n.revision) AS revisions
RETURN src_name, revisions
ORDER BY src_name
"""

graph = db.graph
res1 = graph.run(query1).data()
res2 = graph.run(query2).data()

print("🔸 Revision -> src_names")
for row in res1:
    print(f"{row['revision']} ➜ {row['src_names']}")

print("\n🔸 src_name -> Revisions")
for row in res2:
    print(f"{row['src_name']} ➜ {row['revisions']}")


🔸 Revision -> src_names
7990 ➜ ['CDSILibraryDrawer.UpdateModel()', 'CDSILibraryDrawer._updateSleeve(int)', 'CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem*, CMatrixMxN, ST_SurfaceDataSet*)', 'CDSILibraryDrawer', 'CUIDlgImplantLib._changeSystem(CString)', 'CUIDlgImplantLib.InitSleeve()', 'CActuatorAxialFixture.Fixture_Move(UINT)', 'CActuatorCSFixture.Fixture_Move(UINT)', 'CActuatorHybridFixture.Fixture_Move(UINT)', 'CActuatorPanoFixture.Fixture_Move(UINT)', 'CActuatorTgtFixture.Fixture_Move(UINT)', 'CActuatorCSFixture.OnUpdate(UINT, WPARAM, const CUpdateParam*, const LPVOID)', 'CActuatorTgtFixture.OnUpdate(UINT, WPARAM, const CUpdateParam*, const LPVOID)', 'CActuatorCSFixture.~CActuatorCSFixture()', 'CActuatorCrownDesign.~CActuatorCrownDesign()', 'CActuatorHybridFixture.~CActuatorHybridFixture()', 'CActuatorMPRFixrueBase.~CActuatorMPRFixrueBase()', 'CActuatorMPRFixrueBase.OnAttach()', 'CActuatorMPRFixrueBase.CActuatorMPRFixrueBase(LPCWSTR)', 'CActuatorPanoFixture.~CActuatorPanoFixtu

In [28]:
from oms.dataset.info_base import RelationInfo
# path_trace_map = trace_data.path_trace_map
# rv_path_trace_map = trace_data.rv_path_trace_map
print(len(save_datas))
for idx, data in enumerate(save_datas):
    print(idx, ' : ', data)
    reldata: RelationInfo = data.info_base.relationInfo
    if 0 < len(reldata):
        print(data.info_base.relationInfo)

    print()

90
0  :  7996#CDSILibraryDrawer.UpdateModel()

1  :  7990#CDSILibraryDrawer.UpdateModel()

2  :  7996#CDSILibraryDrawer._updateSleeve(int)

3  :  7990#CDSILibraryDrawer._updateSleeve(int)

4  :  7996#CDSILibraryDrawer._getSleeveColor(int)

5  :  7996#CDSILibraryDrawer._updateFixtureSleeve(int)

6  :  7996#CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem*, CMatrixMxN, ST_SurfaceDataSet*)

7  :  7990#CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem*, CMatrixMxN, ST_SurfaceDataSet*)

8  :  7996#CDSILibraryDrawer

9  :  7990#CDSILibraryDrawer

10  :  8010#CUIDlgImplantLib._changeSystem(CString)

11  :  7990#CUIDlgImplantLib._changeSystem(CString)

12  :  8010#CUIDlgImplantLib.InitSleeve()

13  :  7990#CUIDlgImplantLib.InitSleeve()

14  :  8031#CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem*, CMatrixMxN, ST_SurfaceDataSet*)

15  :  7996#CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem*, CMatrixMxN, ST_SurfaceDataSet*)

16  :  7990#CDSILibraryDrawer.GetShortestIdx(CSurfaceModelItem